### I am running the code again to create the Toronto neighborhoods DF since I am unsure how to use the same DF across multiple notebooks. You can skip down to the bottom of this notebook to see the final DF.

In [15]:
# Importing libraries
import requests
import pandas as pd
import random
import numpy as np
from bs4 import BeautifulSoup

In [16]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [17]:
soup = BeautifulSoup(wiki_url, 'xml')

In [18]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting the table data from wikipedia
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue
        
        
        
        
        
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [19]:
unique_p = set(Postcode)
print('number of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

number of unique Postal codes: 77


In [20]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Borough,Neighbourhood,Postcode
0,East Toronto,The Beaches,M4E
1,West Toronto,"Exhibition Place, Parkdale Village",M6K
2,North York,"Northwood Park, York University",M3J
3,Central Toronto,Forest Hill North,M5P
4,North York,"Emery, Humberlea",M9M


### Now importing the lat and long data for Toronto

In [21]:
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Combining the lat and long DF with the Toronto neighborhoods DF
-First start with changing the Postal Code column name in df_ll to match df_toronto in order to merge the two df's

In [28]:
# Change Postal Code column to match df_toronto Postcode column in order to merge the two df's

df_ll.columns = ['Postcode', 'Latitude', 'Longitude']
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df_toronto.head()

,Borough,Neighbourhood,Postcode
0,East Toronto,The Beaches,M4E
1,West Toronto,"Exhibition Place, Parkdale Village",M6K
2,North York,"Northwood Park, York University",M3J
3,Central Toronto,Forest Hill North,M5P
4,North York,"Emery, Humberlea",M9M


In [30]:
df_final = pd.merge(df_toronto, df_ll, on='Postcode')
df_final

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,West Toronto,"Exhibition Place, Parkdale Village",M6K,43.636847,-79.428191
2,North York,"Northwood Park, York University",M3J,43.767980,-79.487262
3,Central Toronto,Forest Hill North,M5P,43.696948,-79.411307
4,North York,"Emery, Humberlea",M9M,43.724766,-79.532242
5,Downtown Toronto,University of Toronto,M5S,43.662696,-79.400049
6,North York,Hillcrest Village,M2H,43.803762,-79.363452
7,West Toronto,"Little Portugal, Trinity",M6J,43.647927,-79.419750
8,East Toronto,Riverdale,M4K,43.679557,-79.352188
9,East Toronto,India Bazaar,M4L,43.668999,-79.315572
